In [ ]:
import sys
sys.path.append("/Users/yunongliu/Documents/GitHub/IKEA-Manuals-at-Work/src")
import os
from IKEAVideo.dataloader.dataset_keyframe import KeyframeDataset

annotation_file = "./data/data.json"
video_dir = "/Users/yunongliu/Documents/GitHub/ikea-video-cleaning/video"
obj_dir = './data/parts'
manual_img_dir = './data/manual_img'
pdf_dir = './data/pdfs'

dataset = KeyframeDataset(annotation_file, video_dir, transform=None, load_into_mem=False,
                            verbose=False, debug=True, obj_dir=obj_dir, manual_img_dir=manual_img_dir, pdf_dir=pdf_dir)

In [ ]:
video_idx = 0  # Index of the video
frame_idx = 12  # Index of the frame within the video

sample = dataset.__getitem__(video_idx, frame_idx)

In [ ]:
import IKEAVideo.utils.transformations as tra

import trimesh
import numpy as np
mesh_all = sample[0]['meshes'].copy()
scene = trimesh.Scene()
for mesh in mesh_all:
    scene.add_geometry(mesh)
scene.show()

## Get Object Poses in this frame in .glb format

In [ ]:

mesh_all = sample[0]['meshes']
scene = trimesh.Scene()
colors = [
    '#5A9BD5', '#FF6F61', '#77B77A', '#A67EB1', '#FF89B6', '#FFB07B',
    '#C5A3CF', '#FFA8B6', '#A3C9E0', '#FFC89B', '#E58B8B',
    '#A3B8D3', '#D4C3E8', '#66B2AA', '#E4A878', '#6882A4', '#D1AEDD', '#E8A4A6',
    '#A5DAD7', '#C6424A', '#E1D1F4', '#FFD8DC', '#F4D49B', '#8394A8'
]
meshes = None
print(len(mesh_all))
color_index = 0
for m, mesh in enumerate(mesh_all):
    
    # Make the mesh double-sided by duplicating faces with reversed normals
    reversed_faces = mesh.faces[:, ::-1]
    combined_faces = np.vstack((mesh.faces, reversed_faces))
    combined_vertices = np.vstack((mesh.vertices, mesh.vertices))
    mesh = trimesh.Trimesh(vertices=combined_vertices, faces=combined_faces)

    # Assign color to mesh
    color_rgba = trimesh.visual.color.hex_to_rgba(colors[color_index % len(colors)])

    mesh.visual.face_colors = np.tile(color_rgba, (mesh.faces.shape[0], 1))
    color_index += 1
    if meshes is None:
        meshes = mesh
    else:
        meshes = trimesh.util.concatenate(meshes, mesh)

# Rotate the mesh

scene.add_geometry(meshes)
# Export to glb
scene.export('scene.glb')
scene.show()